In [ ]:
# (C) Copyright Aaron Goldberg, 2022.
#
# This code is licensed under the Apache License, Version 2.0. You may
# obtain a copy of this license in the LICENSE.txt file in the root directory
# of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.
#
# Any modifications or derivative works of this code must retain this
# copyright notice, and modified files need to carry a notice indicating
# that they have been altered from the originals.

In [1]:
# Estimate the transmission amplitude parameters based on the aggregated data
import numpy as np
import time
num_shots=1000000
num_jobs=1000
def read_data_from_file(mode_pair, file_counter):
    return np.loadtxt(f'TMSV_data/{mode_pair}_{file_counter}.csv', delimiter=',',dtype ='int')#read back as integers

pair_ID='04'
my_bounds=((0.1,0.9),(0.1,0.9))
my_guess=[0.65,0.65]


maxval=8
totalcounts04=np.zeros((maxval+1,maxval+1))

start = time.time()
for job_counter in range(num_jobs):
    totalcounts04=totalcounts04+ read_data_from_file(pair_ID, job_counter)
end = time.time()
print('timing:',end - start)

timing: 0.8779323101043701


In [4]:
#Now do KL divergence but normalize the probability distributions to only include terms that can be measured, as per https://iopscience.iop.org/article/10.1088/1367-2630/10/4/043022/pdf
#This normalization won't change much because the probability of not being registered by the PNRDs is tiny
import scipy
import scipy.special
from scipy.optimize import minimize 
from scipy.optimize import LinearConstraint 
from scipy import stats


upper_state_cutoff=25 # This does need to change because even a large |nn> has a chance of losing enough photons to look like 0 to 9 photons remaining, so it adds a tiny bit to the overall probability of detecting that many photons. At least three sig figs remain the same when going from 20 to 25, so 25 should be sufficient
r=1.0
state_coefficients_squared=[np.tanh(r)**(2*n)/np.cosh(r)**2 for n in range(upper_state_cutoff)]

    
def total_error04KL(params):
    # Find the KL divergence in the probability distribution as a function of the guessed error value from the ideal case
    eta1=params[0]
    eta2=params[1]
    my_probabilities=np.array([[eta1**(2*M)*eta2**(2*N)*(1-eta1**2)**(-M)*(1-eta2**2)**(-N)
                                *sum([state_coefficients_squared[m]*scipy.special.binom(m, M)*scipy.special.binom(m, N)*((1-eta1**2)*(1-eta2**2))**(m) for m in range(max(M,N),upper_state_cutoff)])
                                     for N in range(maxval+1)] for M in range(maxval+1)])
    my_probabilities=my_probabilities/my_probabilities.sum()#This line seems to make a difference only at six sig figs
    KL_divergence=scipy.stats.entropy(counts04.flatten(),qk=my_probabilities.flatten()) # Second argument is for the "theory" - we want the ideal "theory" for this experiment
    return(KL_divergence)
counts04=totalcounts04
results=scipy.optimize.minimize(total_error04KL,x0=my_guess, bounds=my_bounds).x

In [5]:
print('Estimated transmission amplitudes for modes 0 and 4 without normalization in KL divergence: ',results)

Estimated transmission amplitudes for modes 0 and 4 without normalization in KL divergence:  [0.5497347  0.58129087]


In [6]:
#This time, 3 parameters: loss in the pump beam, if it is a coherent state, does r-> eta0 r
#Now these results are wild: the KL divergence becomes smaller by an order of magnitude if we pretend that the pump mode was
#***amplified*** by a factor of 1.5 and there was just a lot more loss in the later arms
import scipy
import scipy.special
from scipy.optimize import minimize 
from scipy.optimize import LinearConstraint 
from scipy import stats


upper_state_cutoff=100 # This does need to change because even a large |nn> has a chance of losing enough photons to look like 0 to 9 photons remaining, so it adds a tiny bit to the overall probability of detecting that many photons. At least three sig figs remain the same when going from 20 to 25, so 25 should be sufficient
r=1.0
    
def total_error04KL3(params):
    # Find the KL divergence in the probability distribution as a function of the guessed error value from the ideal case
    eta1=params[0]
    eta2=params[1]
    eta3=params[2]
    state_coefficients_squared=[np.tanh(eta3*r)**(2*n)/np.cosh(eta3*r)**2 for n in range(upper_state_cutoff)]
    my_probabilities=np.array([[eta1**(2*M)*eta2**(2*N)*(1-eta1**2)**(-M)*(1-eta2**2)**(-N)
                                *sum([state_coefficients_squared[m]*scipy.special.binom(m, M)*scipy.special.binom(m, N)*((1-eta1**2)*(1-eta2**2))**(m) for m in range(max(M,N),upper_state_cutoff)])
                                     for N in range(maxval+1)] for M in range(maxval+1)])
    my_probabilities=my_probabilities/my_probabilities.sum()#This line seems to make a difference only at six sig figs
    KL_divergence=scipy.stats.entropy(counts04.flatten(),qk=my_probabilities.flatten()) # Second argument is for the "theory" - we want the ideal "theory" for this experiment
    return(KL_divergence)

my_bounds=((0.1,0.9),(0.1,0.9),(0.3,3.0))
my_guess=[0.65,0.65,0.75]
counts04=totalcounts04
results=scipy.optimize.minimize(total_error04KL3,x0=my_guess, bounds=my_bounds)

In [8]:
print('Estimated transmission amplitudes for modes 0 and 4 with normalization in KL divergence: ',results.x,' with KL divergence: ',results.fun,' (if we restrict to r=1 then KL divergence is 0.02100804924676686)')

Estimated transmission amplitudes for modes 0 and 4 with normalization in KL divergence:  [0.28349775 0.29979221 1.60183387]  with KL divergence:  0.0019409178656929534  (if we restrict to r=1 then KL divergence is 0.02100804924676686)


In [9]:
#This time, 5 parameters: add the ability for dark counts. After loss, derive from https://doi.org/10.1103/PhysRevA.85.023820 
#p_k(dark, loss)=sum_{m=0}^k p_m(loss) exp(-v)v^(k-m)/(k-m)!, where v is dark count rate, for each mode
#When I checked an empty run the dark count rate was about 0.02 on average, up to 0.045 for one of the detectors
#Now the algorithm gives much better KL divergences if dark counts are very high, like 5-8%. 
import scipy
import scipy.special
from scipy.optimize import minimize 
from scipy.optimize import LinearConstraint 
from scipy import stats


upper_state_cutoff=100 # This does need to change because even a large |nn> has a chance of losing enough photons to look like 0 to 9 photons remaining, so it adds a tiny bit to the overall probability of detecting that many photons. At least three sig figs remain the same when going from 20 to 25, so 25 should be sufficient
r=1.0
#state_coefficients_squared=[np.tanh(r)**(2*n)/np.cosh(r)**2 for n in range(upper_state_cutoff)]

    
def total_error04KL5(params):
    # Find the KL divergence in the probability distribution as a function of the guessed error value from the ideal case
    eta1=params[0]
    eta2=params[1]
    eta3=params[2]
    v1=params[3]
    v2=params[4]
    state_coefficients_squared=[np.tanh(eta3*r)**(2*n)/np.cosh(eta3*r)**2 for n in range(upper_state_cutoff)]
    my_probabilities=np.array([[eta1**(2*M)*eta2**(2*N)*(1-eta1**2)**(-M)*(1-eta2**2)**(-N)
                                *sum([state_coefficients_squared[m]*scipy.special.binom(m, M)*scipy.special.binom(m, N)*((1-eta1**2)*(1-eta2**2))**(m) for m in range(max(M,N),upper_state_cutoff)])
                                     for N in range(maxval+1)] for M in range(maxval+1)])
    my_darkened_probs=np.array([[np.exp(-v1-v2)*np.sum([[v1**(m-k)*v2**(n-l)*my_probabilities[k,l]/scipy.special.factorial(m-k)/scipy.special.factorial(n-l) for k in range(m+1)]for l in range(n+1)]) 
                                 for n in range(maxval+1)]for m in range(maxval+1)])
    my_probabilities=my_darkened_probs/my_darkened_probs.sum()
    #my_probabilities=np.array([[(eta1**2/(1-eta1**2))**j*np.exp(-v1)*(eta2**2/(1-eta2**2))**k*np.exp(-v2)*sum([state_coefficients_squared[n]*((1-eta1**2)*(1-eta2**2))**n*scipy.special.genlaguerre(j,n-j)(v1*(1-1/eta1**2))*scipy.special.genlaguerre(k,n-k)(v2*(1-1/eta2**2)) for n in range(max(j,k),upper_state_cutoff)]) for j in range(maxval+1)] for k in range (maxval+1)])
    #my_probabilities=my_probabilities/my_probabilities.sum() #This method seems more efficient but takes longer
    KL_divergence=scipy.stats.entropy(counts04.flatten(),qk=my_probabilities.flatten()) # Second argument is for the "theory" - we want the ideal "theory" for this experiment
    return(KL_divergence)

my_bounds=((0.1,0.9),(0.1,0.9),(0.3,3.0),(0.001,0.2),(0.001,0.2))
my_guess=[0.65,0.65,0.75,0.02,0.02]
counts04=totalcounts04
results=scipy.optimize.minimize(total_error04KL5,x0=my_guess, bounds=my_bounds)

In [10]:
print('Estimated transmission amplitudes, squeezing parameter, and dark count rates for modes 0 and 4 with normalization in KL divergence: ',results.x,' with KL divergence: ',results.fun,' (if we restrict to r=1 then KL divergence is 0.02100804924676686); if we restrict to no dark counts but varying r then KL divergence is  0.0019409178656531918')

Estimated transmission amplitudes, squeezing parameter, and dark count rates for modes 0 and 4 with normalization in KL divergence:  [0.38206446 0.39202352 1.29995433 0.03418712 0.06568094]  with KL divergence:  0.00014556573932841924  (if we restrict to r=1 then KL divergence is 0.02100804924676686); if we restrict to no dark counts but varying r then KL divergence is  0.0019409178656531918
